# Testing IEEE 802.15.4 packet sink block Python translation

In [1]:
def count_set_bits(n: int, bits: int = 32) -> int:
    """Return the number of set bits in the lowest 'bits' of 'n'."""
    mask = (1 << bits) - 1  # Create a mask for the lowest 'bits' bits
    return bin(n & mask).count("1")


print(
    count_set_bits(0b0100_1110_0000_0111_0111_1010_1110_0110, 4)
)  # Counts bits in the lowest 4 bits (0110) → Output: 2

2


In [2]:
import numpy as np

# Chip mapping for MSK-based IEEE 802.15.4 demodulation
# Important: sequences are inverted bit-wise. The LSB of each uint32 number
# is the last chip sent on air
CHIP_MAPPING: np.ndarray = np.array(
    [
        0xE077AE6C,  # 0
        0xCE077AE6,  # 1
        0x6CE077AE,  # 2
        0xE6CE077A,  # 3
        0xAE6CE077,  # 4
        0x7AE6CE07,  # 5
        0x77AE6CE0,  # 6
        0x877AE6CE,  # 7
        0x1F885193,  # 8
        0x31F88519,  # 9
        0x931F8851,  # A
        0x1931F885,  # B
        0x51931F88,  # C
        0x851931F8,  # D
        0x8851931F,  # E
        0x78851931,  # F
    ],
    dtype=np.uint32,
)
MAX_PKT_LEN: int = 127


def decode_chips(chips32: int, threshold: int = 32) -> int:
    """Decodes the received chip sequence by comparing it against a known mapping."""
    best_match = 0xFF
    min_threshold = 33  # Value greater than the maximum possible errors (32 bits)

    for i in range(16):
        # 0x7FFFFFFE masks out the first and last bit, since these depend on previous chip data
        # This is because we are using differential encoding
        masked_diff = (chips32 ^ CHIP_MAPPING[i]) & 0x7FFFFFFE
        diff_bits = count_set_bits(masked_diff, 32)  # Count the number of bits that differ
        print(f"{diff_bits = }")

        if diff_bits < min_threshold:
            best_match = i
            min_threshold = diff_bits

    if min_threshold <= threshold:
        return best_match & 0xF  # Return position in chip mapping

    return 0xFF  # If no valid match was found, return 0xFF to indicate an error.


# Use threshold = 32 to simply find the closest match!
# Use low threshold for preamble detection and length reading
# Use high threshold for payload (we are sure we received a IEEE 802.15.4 packet, but may be interfered during the payload)
test_variable = decode_chips(0b0111_1110_1111_1111_1111_1111_1111_1111, 32)
print(f"{test_variable = }")

diff_bits = 12
diff_bits = 12
diff_bits = 11
diff_bits = 12
diff_bits = 13
diff_bits = 12
diff_bits = 13
diff_bits = 13
diff_bits = 18
diff_bits = 18
diff_bits = 19
diff_bits = 18
diff_bits = 17
diff_bits = 18
diff_bits = 17
diff_bits = 17
test_variable = 2


In [3]:
def map_nibbles_to_chip_string(byte_array: list[int], chip_mapping: np.ndarray) -> str:
    result = []
    for byte in byte_array:
        for nibble in [byte & 0x0F, (byte >> 4) & 0x0F]:  # Extract LSB first, then MSB
            mapped_value = chip_mapping[nibble]
            binary_string = f"{mapped_value:032b}"  # Convert to 32-bit binary
            result.append(binary_string)  # Append delimiter
    return "_".join(result)


# Example usage:
byte_array = [0x00, 0x00, 0x00, 0x00, 0xA7]
print(map_nibbles_to_chip_string(byte_array, CHIP_MAPPING))

11100000011101111010111001101100_11100000011101111010111001101100_11100000011101111010111001101100_11100000011101111010111001101100_11100000011101111010111001101100_11100000011101111010111001101100_11100000011101111010111001101100_11100000011101111010111001101100_10000111011110101110011011001110_10010011000111111000100001010001
